In [1]:
import pandas as pd 
from pathlib import Path
import numpy as np
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [6]:
# Read in the CSV file as a Pandas Dataframe
df = pd.read_csv(Path("resources/baseballData.csv"))
#ridding csv of partial seasons
dfFullYear = df[~df['year'].isin([2020, 2024])]
#dropping unesscessary columns
#'babip','bacon',
dfFullYear.drop(columns=["player_id",'player_age', "last_name, first_name", 'year', 'ab', 'pa', 'hit', 'single', 'double', 'triple', 'out_zone_swing_miss', 'babip','bacon',
       'out_zone_swing', 'out_zone_percent', 'out_zone', 'in_zone_swing_miss', 'single', 'double', 'triple', 'home_run', 'strikeout', 'walk', 'bb_percent', 'on_base_percent', 'on_base_plus_slg', 'isolated_power','woba','wobacon',
       'in_zone_swing','batted_ball', 'groundballs', 'flyballs', 'linedrives', 'popups',
       'home_run', 'strikeout', 'walk'], inplace=True)
dfFullYear.columns
dfFullYear.head(10)

/var/folders/q8/t_k5gtj11z5dpr8r51m6nbz00000gn/T/ipykernel_84724/1161540600.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfFullYear.drop(columns=["player_id",'player_age', "last_name, first_name", 'year', 'ab', 'pa', 'hit', 'single', 'double', 'triple', 'out_zone_swing_miss', 'babip','bacon',


,k_percent,batting_avg,slg_percent,exit_velocity_avg,launch_angle_avg,sweet_spot_percent,barrel,barrel_batted_rate,solidcontact_percent,flareburner_percent,...,whiff_percent,swing_percent,pull_percent,straightaway_percent,opposite_percent,f_strike_percent,groundballs_percent,flyballs_percent,linedrives_percent,popups_percent
0,18.5,0.240,0.409,88.5,10.8,28.5,21,5.0,5.2,21.6,...,23.1,53.4,34.7,40.6,24.5,64.0,49.4,22.6,20.9,7.1
1,15.5,0.273,0.553,93.0,15.7,34.8,58,13.1,8.4,24.9,...,23.2,44.7,41.9,36.4,21.7,52.1,37.6,25.6,28.7,8.1
2,23.4,0.250,0.486,91.3,12.2,31.4,42,10.9,8.1,20.3,...,32.0,43.9,38.4,43.9,16.9,60.3,43.6,24.9,24.9,6.5
3,13.2,0.246,0.423,87.4,15.8,33.5,23,5.6,6.6,23.3,...,17.9,52.9,36.9,38.3,24.8,65.5,37.6,24.5,29.1,8.7
4,10.5,0.287,0.453,89.5,12.6,35.7,28,5.5,7.8,26.3,...,16.8,48.1,35.3,41.4,23.3,61.9,42.4,18.4,31.6,7.6
5,16.0,0.276,0.471,90.6,15.6,34.1,23,5.8,7.5,27.3,...,18.1,45.4,40.6,38.3,19.3,57.3,36.8,26.6,28.1,8.5
6,15.3,0.224,0.358,85.7,16.2,32.1,11,2.5,4.2,23.8,...,18.1,45.0,46.0,32.6,21.2,56.5,41.3,24.7,24.2,9.7
7,10.9,0.244,0.480,90.7,14.2,29.5,41,7.7,5.3,22.7,...,16.3,45.4,43.3,37.0,19.5,56.0,42.8,25.5,23.8,7.9
8,26.7,0.247,0.453,86.3,12.1,33.9,31,8.5,5.5,21.9,...,31.2,60.2,36.6,35.8,27.6,70.4,43.7,22.7,28.7,4.9
9,16.8,0.265,0.380,89.5,4.4,33.3,18,3.7,8.0,28.6,...,16.8,40.7,28.2,39.7,32.1,57.2,57.6,13.4,27.2,1.9


In [7]:
#Scale the Data
scaler = StandardScaler()
scaler.fit(dfFullYear)

StandardScaler()

In [14]:
#Assign Variables to be Trained
X = dfFullYear.drop('batting_avg', axis=1)
y = dfFullYear["batting_avg"]

In [15]:
#Split the data into training & testing
X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.2, random_state = 50)


In [17]:
#Create a Rabndom Forest Regressor Model
rf_model = RandomForestRegressor(n_estimators=100, random_state=50)

In [18]:
#Train the Model
rf_model.fit(X_train, y_train)

RandomForestRegressor(random_state=50)

In [19]:
#Make Predictions
predictions = rf_model.predict(X_test)
print(predictions)


[0.28735 0.29106 0.22922 0.29312 0.29351 0.25358 0.2676  0.27383 0.22823
 0.27082 0.29692 0.27276 0.28171 0.27078 0.2519  0.24717 0.27972 0.27584
 0.2861  0.27987 0.26374 0.25743 0.27713 0.23814 0.25328 0.30887 0.24792
 0.25431 0.2579  0.25868 0.27666 0.27597 0.24371 0.27964 0.26662 0.25535
 0.25375 0.23349 0.28004 0.25058 0.2709  0.24638 0.26947 0.31611 0.27068
 0.26682 0.26969 0.29477 0.27444 0.25096 0.25199 0.27778 0.24714 0.24754
 0.27761 0.27606 0.29141 0.23979 0.25481 0.26788 0.2874  0.29187 0.27678
 0.26927 0.26224 0.28894 0.24461 0.2461  0.28504 0.24723 0.28081 0.22756
 0.2516  0.26137 0.25937 0.23653 0.26897 0.24555 0.23975 0.2867  0.26182
 0.25022 0.25979 0.26284 0.27745 0.27239 0.28172 0.25652 0.27098 0.27478
 0.28264 0.2619  0.25741 0.23541 0.23963 0.30261 0.25737 0.28571 0.31641
 0.25537 0.23936 0.26348 0.30394 0.30364 0.2569  0.25347 0.24358 0.26621
 0.26903 0.28792 0.28434 0.25904 0.31109 0.27405 0.25571 0.30521 0.25147
 0.22891 0.28271 0.27864 0.28363 0.26577 0.29413 0.

In [20]:
mean = mean_absolute_error(y_test, predictions)
print("Mean", mean)

Mean 0.011427420814479646
